In [11]:
PERIOD = 7

# Env
import gym, json
from gym import spaces
from epipolicy.core.epidemic import construct_epidemic
from epipolicy.obj.act import construct_act
import numpy as np
import os
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt

# Epipolicy gym environment
class EpiEnv(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['human']}

    def __init__(self, session):
        super(EpiEnv, self).__init__()
        self.epi = construct_epidemic(session) # this line is taking minutes!!
        total_population = np.sum(self.epi.static.default_state.obs.current_comp)
        obs_count = self.epi.static.compartment_count * self.epi.static.locale_count * self.epi.static.group_count
        action_count = 0
        action_param_count =  0
        for itv in self.epi.static.interventions:
            if not itv.is_cost:
                action_count += 1
                action_param_count += len(itv.cp_list)
        self.act_domain = np.zeros((action_param_count, 2), dtype=np.float64)
        index = 0
        for itv in self.epi.static.interventions:
            if not itv.is_cost:
                for cp in itv.cp_list:
                    self.act_domain[index, 0] = cp.min_value
                    self.act_domain[index, 1] = cp.max_value
                    index += 1
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Box(low=0, high=1, shape=(action_count,), dtype=np.float64)
        # Example for using image as input:
        self.observation_space = spaces.Box(low=0, high=total_population, shape=(obs_count,), dtype=np.float64)

    def step(self, action):
        expanded_action = np.zeros(len(self.act_domain), dtype=np.float64)
        index = 0
        for i in range(len(self.act_domain)):
            if self.act_domain[i, 0] == self.act_domain[i, 1]:
                expanded_action[i] = self.act_domain[i, 0]
            else:
                expanded_action[i] = action[index]
                index += 1

        epi_action = []
        index = 0
        for itv_id, itv in enumerate(self.epi.static.interventions):
            if not itv.is_cost:
                epi_action.append(construct_act(itv_id, expanded_action[index:index+len(itv.cp_list)]))
                index += len(itv.cp_list)

        total_r = 0
        for i in range(PERIOD):
            state, r, done = self.epi.step(epi_action)
            total_r += r
            if done:
                break
        return state.obs.current_comp.flatten(), total_r, done, dict()

    def reset(self):
        state = self.epi.reset()
        return state.obs.current_comp.flatten()  # reward, done, info can't be included
    def render(self, mode='human'):
        # if not hasattr(self, 'fig'):  # Create a plot the first time
        #     self.fig, self.ax = plt.subplots()
        # self.ax.clear()

        # Assuming `self.epi.state` contains the current state of compartments
        state = self.epi.get_current_state()
        state = state.obs.current_comp.flatten()
        print(state)
    def close(self):
        pass

In [18]:
from tqdm import tqdm

epi_ids = ["SIR_A"] #, "SIR_B", "SIRV_A", "SIRV_B", "COVID_A", "COVID_B"]
TOTAL_POP = 2224526
MAX_DOSES = 9200

def vaccine_rate(total_pop, pc_pop, nmonths, max_doses):
    return total_pop * pc_pop / (nmonths * 30) / max_doses 

# Executing baseline policy
def run(scenario, baseline, seed=None): 
    # pbar = tqdm()
    scenario = json.loads(open('jsons/'+scenario+'.json', "r").read()) 
    env = EpiEnv(scenario) 
    if seed: 
        np.random.seed(seed)

    obs = env.reset()
    done = False
    total_r = 0
    i = 0
    while not done:
        # Random Strategy 
        if baseline == 'random':
            action = np.random.uniform(0, 1, env.action_space.shape[0])
        # Lax Strategy 
        elif baseline == 'lax': 
            lax_rate = vaccine_rate(TOTAL_POP, 0.7, 12, MAX_DOSES) # 70% population in 12 months
            action = np.zeros(env.action_space.shape[0])
            action[0] = lax_rate
            # 100% mask compliance
            action[1] = 1 
        # Aggressive Strategy 
        elif baseline == 'agg': 
            agg_rate = vaccine_rate(TOTAL_POP, 0.85, 9, MAX_DOSES) # 85% population in 9 months
            action = np.zeros(env.action_space.shape[0]) 
            if i <= 9*30:
                action[0] = agg_rate
            # 80% mask compliance
            action[1] = 0.8
            # Closure for the first 120 days
            if i <= 4*30:
                action[2:] = 1
            else:
                # Relaxation afterwards
                action[2:] = 0.5
        obs, r, done, info = env.step(action)
        total_r += r
        i += 1
        # pbar.update(1)
        env.render()
    # pbar.close()
    return total_r

In [21]:
from tqdm import tqdm
lax = [run(scenario, "lax") for scenario in (epi_ids)]
# agg = [run(scenario, "agg") for scenario in tqdm(epi_ids)]
# rdm = [[run(scenario, "random", seed) for scenario in epi_ids] for seed in range(4)]
import ast
# lax = ast.literal_eval("[-104445644.46512368, -104192852.73258403, -122356909.00286072, -115621647.49235287, -167601735.19676903, -166561829.1777328]")
# agg = ast.literal_eval("[-135913288.65608233, -1576018846.0933764, -166295695.67466035, -1577127251.903828, -139872331.12360632, -742866221.3805492]")
# rdm = ast.literal_eval("[[-94098856.12817216, -737008342.4036739, -1008273720.9517195, -884378580.353922, -559266476.5680647, -408595095.6950983], [-101626164.0779075, -820907206.9828014, -3600618677.002645, -843609324.7176187, -1365342706.749666, -444279408.64855975], [-144693312.09686247, -730451739.1314715, -3473763372.677735, -802170501.0551431, -869570338.9482585, -421032165.6451399], [-101192853.15949182, -693508291.0335313, -3110726459.3301916, -717404868.1967467, -826728617.4729577, -387690550.8985477]]")
rdm = []

# for i, sce in  (enumerate(epi_ids)):
#     best_r = -1e100
#     for seed in rdm:
#         best_r = max(best_r, seed[i])
#     print(sce, best_r)

[2.19407088e+06 9.11596160e+01 3.03639566e+04]
[2.16373176e+06 7.80646236e+01 6.07161769e+04]
[2.13340235e+06 6.61840404e+01 9.10574664e+04]
[2.10308154e+06 5.55521435e+01 1.21388906e+05]
[2.07276825e+06 4.61634288e+01 1.51711588e+05]
[2.04246144e+06 3.79792158e+01 1.82026585e+05]
[2.01216014e+06 3.09346698e+01 2.12334926e+05]
[1.98186348e+06 2.49457899e+01 2.42637571e+05]
[1.95157068e+06 1.99159913e+01 2.72935403e+05]
[1.92128104e+06 1.57420031e+01 3.03229215e+05]
[1.89099397e+06 1.23188980e+01 3.33519712e+05]
[1.86070895e+06 9.54416336e+00 3.63807505e+05]
[1.83042556e+06 7.32079536e+00 3.94093117e+05]
[1.80014345e+06 5.55946852e+00 4.24376993e+05]
[1.76986232e+06 4.17987267e+00 4.54659499e+05]
[1.73958195e+06 3.11134179e+00 4.84940940e+05]
[1.70930215e+06 2.29291195e+00 5.15221559e+05]
[1.67902277e+06 1.67294665e+00 5.45501552e+05]
[1.64874372e+06 1.20845899e+00 5.75781074e+05]
[1.61846489e+06 8.64245033e-01 6.06060244e+05]
[1.58818623e+06 6.11923379e-01 6.36339155e+05]
[1.55790769e+

In [5]:
print(lax,agg)

[-104445644.4651237] [-135913288.65608233]


In [12]:
def plot_learning_curve(scenario):
    cut_step = 30000
    sid = epi_ids.index(scenario)
    data = {
        "timesteps": [],
        "rewards": [],
        "methods": []
    }

    for i in range(292, cut_step, 292):
        data["timesteps"].append(i)
        data["rewards"].append(lax[sid])
        data["methods"].append("Lax")

    for i in range(292, cut_step, 292):
        data["timesteps"].append(i)
        data["rewards"].append(agg[sid])
        data["methods"].append("Aggressive")

    for j in range(len(rdm)):
        for i in range(292, cut_step, 292):
            data["timesteps"].append(i)
            data["rewards"].append(rdm[j][sid])
            data["methods"].append("Random")

    for folder in os.listdir("runs"):
        if not os.path.isfile(folder):
            tokens = folder.split("__")
            if len(tokens) == 4:
                method = tokens[1]
                if method == "SAC":
                    continue
                elif method == "SAC_scale":
                    method = "SAC"
                if scenario == tokens[0]:
                    f = open('runs/{}/records.csv'.format(folder), 'r')
                    for line in f:
                        t, total_r, _ = line.split('|')
                        t = int(t)
                        if t > cut_step:
                            break
                        total_r = float(total_r)
                        data["timesteps"].append(t)
                        data["rewards"].append(total_r)
                        data["methods"].append(method)
                    f.close()
    df = pd.DataFrame(data)
    palette = {"Lax": "black", "Aggressive":"red", "Random":"gray", "PPO":"blue", "SAC":"green"}
    g = sns.lineplot(x="timesteps", estimator="mean", y="rewards", hue="methods", palette=palette, ci=80, n_boot=10, data=df, hue_order=["Lax", "Aggressive", "Random", "PPO", "SAC"])
    g.set(yscale='symlog')
    size = 15
    g.set_xlabel("timesteps", fontsize = size) # X label
    g.set_ylabel("rewards", fontsize = size) # Y label
    g.get_legend().remove()
    for label in (g.get_xticklabels() + g.get_yticklabels()):
        label.set_fontsize(14)
#     plt.legend(loc="lower left", ncol=5)
#     pos = g.get_position()
#     g.set_position([pos.x0, pos.y0, pos.width, pos.height * 0.85])
#     g.legend(
#         loc='upper center', 
#         bbox_to_anchor=(0.5, 1.35),
#         ncol=5, 
#     )
    plt.savefig("plots/{}_learning_curve.png".format(scenario), bbox_inches="tight")
    plt.clf()

In [22]:
for scenario in epi_ids:
    plot_learning_curve(scenario)

FileNotFoundError: [Errno 2] No such file or directory: 'runs/SIR_A__PPO__0__1733527763/records.csv'

In [18]:
import matplotlib.transforms as mtrans
def plot_intervention(scenario, method):
    interventions = ["Vaccination", "Mask-wearing", "School closure", "Workplace closure"]
    colors = ["blue", "green", "black", "red"]
    best_actions = None
    best_total_r = -1e100
    for folder in os.listdir("runs"):
        if not os.path.isfile(folder):
            tokens = folder.split("__")
            if len(tokens) == 4:
                if scenario == tokens[0] and method == tokens[1]:
                    f = open('runs/{}/records.csv'.format(folder), 'r')
                    for line in f:
                        t, total_r, actions = line.split('|')
                        t = int(t)
                        total_r = float(total_r)
                        if best_total_r < total_r:
                            best_total_r = total_r
                            best_actions = ast.literal_eval(actions)
                    f.close()
    X = [i for i in range(0, len(best_actions) * 7, 7)]
    Ys = np.array(best_actions).transpose()
    for i, Y in enumerate(Ys):
        if i != 3:
            plt.plot(X, Y, label=interventions[i], color=colors[i])
        else:
            plt.plot(X[::2], Y[::2], '.', color=colors[i], label=interventions[i])
    plt.xlabel("Days", fontsize=15)
    plt.ylabel("Control parameter", fontsize=15)
    plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=14)
    plt.ylim(-0.1,1.1)
    plt.legend()
#     plt.legend(loc="lower left", ncol=5)
#     ax = plt.subplot(111)
#     pos = ax.get_position()
#     ax.set_position([pos.x0, pos.y0, pos.width, pos.height * 0.85])
#     plt.legend(
#         loc='upper center', 
#         bbox_to_anchor=(0.5, 1.35),
#         ncol=4, 
#     )
    plt.savefig("plots/{}_intervention_by_{}.png".format(scenario, method), bbox_inches="tight")
    plt.clf()
    return best_total_r

In [19]:
for scenario in epi_ids:
    print(scenario, plot_intervention(scenario, "PPO"))

FileNotFoundError: [Errno 2] No such file or directory: 'runs/SIR_A__PPO__0__1733527763/records.csv'

In [9]:
for scenario in epi_ids:
    print(scenario, plot_intervention(scenario, "SAC_scale"))

SIR_A -43423086.54567317
SIR_B -73352017.9536704
SIRV_A -99873613.56467989
SIRV_B -169171358.74782342
COVID_A -65551435.815168954
COVID_B -128619595.43974368


<Figure size 432x288 with 0 Axes>

In [10]:
def compute_perf(method):
    data = [[] for i in range(len(epi_ids))]
    if method == "random":
        for j in rdm:
            for i, v in enumerate(j):
                data[i].append(v)
    else:
        for folder in os.listdir("runs"):
            if not os.path.isfile(folder):
                tokens = folder.split("__")
                if len(tokens) == 4:
                    if method == tokens[1]:
                        sid = epi_ids.index(tokens[0])
                        f = open('runs/{}/records.csv'.format(folder), 'r')
                        best_total_r = -1e100
                        for line in f:
                            t, total_r, actions = line.split('|')
                            t = int(t)
                            if t > 32000:
                                break
                            total_r = float(total_r)
                            if best_total_r < total_r:
                                best_total_r = total_r
                        data[sid].append(best_total_r)
                        f.close()
    mean = []
    std = []
    for i in range(len(epi_ids)):
        mean.append(np.mean(data[i]))
        std.append(np.std(data[i]))
    return mean, std

In [20]:
latex = """\\begin{table}[htb]
\\begin{tabular}{ |p{1.5cm}||p{2.5cm}|p{2.5cm}|}
 \hline
 Algorithms & \\textbf{SIR-A} & \\textbf{SIR-B}\\\\
\hline
Lax& $@$&$@$\\\\
\hline
Aggressive& $@$&$@$\\\\
\hline
Random& $@ \pm @$&$@ \pm @$\\\\
\hline
PPO& $@ \pm @$&$@ \pm @$\\\\
  \hline
SAC& $@ \pm @$&$@ \pm @$\\\\
  \hline
 \hline
  & \\textbf{SIRV-A} & \\textbf{SIRV-B}\\\\
\hline
Lax& $@$&$@$\\\\
\hline
Aggressive& $@$&$@$\\\\
\hline
Random& $@ \pm @$&$@ \pm @$\\\\
\hline
PPO& $@ \pm @$&$@ \pm @$\\\\
  \hline
SAC& $@ \pm @$&$@ \pm @$\\\\
  \hline
 \hline
  & \\textbf{Covid19-A} & \\textbf{Covid19-B}\\\\
\hline
Lax& $@$&$@$\\\\
\hline
Aggressive& $@$&$@$\\\\
\hline
Random& $@ \pm @$&$@ \pm @$\\\\
\hline
PPO& $@ \pm @$&$@ \pm @$\\\\
  \hline
SAC& $@ \pm @$&$@ \pm @$\\\\
  \hline
\end{tabular}
\caption{Mean and standard deviation of the best performance of each algorithm across different random seeds}
\label{table:best}
\end{table}"""

In [21]:
def str_number(v):
    tmp = "{:.2e}".format(v)
    a, b = tmp.split("e")
    b = int(b)
    return a + " \\times 10^{" + str(b) + "}"
means = []
stds = []
li = []
new_latex = latex
for method in ["random", "PPO", "SAC_scale"]:
    mean, std = compute_perf(method)
    means.append(mean)
    stds.append(std)
for i in range(3):
    ll = []
    for j in range(2):
        sid = j + i*2
        new_latex = new_latex.replace("@", str_number(lax[sid]), 1)
        ll.append(lax[sid])
    li.append(ll)
    ll = []
    for j in range(2):
        sid = j + i*2
        new_latex = new_latex.replace("@", str_number(agg[sid]), 1)
        ll.append(agg[sid])
    li.append(ll)
    for k in range(3):
        ll = []
        for j in range(2):
            sid = j + i*2
            new_latex = new_latex.replace("@", str_number(means[k][sid]), 1)
            new_latex = new_latex.replace("@", str_number(stds[k][sid]), 1)
            ll.append(means[k][sid])
            ll.append(stds[k][sid])
        li.append(ll)
print(new_latex)

NameError: name 'compute_perf' is not defined

In [16]:
for i, l in enumerate(li):
    if i % 5 == 3:
        print(l)

[-43261307.390957445, 202418.96571379126, -42936909.6037267, 702527.5157677324]
[-83407644.31701201, 1193689.4926395481, -81238681.0061254, 1523535.3145508517]
[-45705731.05135028, 472945.38396688225, -46170694.37717085, 520757.2242022697]
